# Tableau Project - Videogame Co.

## Where do we open our new office?

### Instrucciones
#### Somos una compañía de Videogames de 50 empleados. Tenemos que abrir una oficina nueva, pero, ¿dónde?
#### Contamos con 20 desarrolladores, 20 UX/UI y 10 ejecutivos.
#### Tiene que haber alrededor:
#### 1-. Compañías de Software
#### 2-. Buen ratio entre grades compañías y start-ups
#### 3-. Empresas de diseño, diseño web, relacionado con UX
#### 4-. Restaurantes para que los ejecutivos puedan reunirse y hacer negocios 
#### 5-. Evitar compañías muy viejas alrededor

### Variables interesantes que mirar
#### Total money raised (inversiones), competitors, latitud y longitud de oficinas, twitter (influencia de la empresa, API tweety), cathegory code

In [9]:
import re
import pandas as pd
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies
from pandas.io.json import json_normalize

In [50]:
# Para evitar compañías viejas alrededor, vamos a considerar qué es una compañía vieja, respondiendo a: ¿En qué año se fundaron
# las compañías más importantes de tecnología hoy en día? Google en el 1998, Intel 1968, Microsoft 1975, Apple 1976
# Tenemos en cuenta las compañías más importantes, ordenadas por valuation amount descending.

#Primero preguntamos a mongo por 1000 compañías de Software para sacar su lat y long.
#Software Co

cursor1 = db.companies.find({"$and": [{"category_code": 'software'}, {"offices.latitude": {"$not": re.compile("^N.*")}}, {"founded_year": {"$gte": 1965}} ]}).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(1000)

software_co = list(map(lambda x: [x["name"], x["offices"], x["category_code"]], list(cursor1)))

software = pd.DataFrame(software_co)

In [51]:
#También preguntamos a mongo por 500 compañías de Mobile para sacar su lat y long, nos interesa tener empresas de movil cerca para sacar videojuegos en el movil.
#Mobile Co

cursor2 = db.companies.find({"$and": [{"category_code": 'mobile'}, {"offices.latitude": {"$not": re.compile("^N.*")}}, {"founded_year": {"$gte": 1965}} ]}).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(500)

mobile_co = list(map(lambda x: [x["name"], x["offices"], x["category_code"]], list(cursor2)))

mobile = pd.DataFrame(mobile_co)

In [52]:
#También nos interesa saber dónde está la competencia, ya que significa que hay mercado en esa zona. 
#Preguntamos a mongo por 1000 compañías de VideoGames para sacar su lat y long.
#VideoGames Co

cursor3 = db.companies.find({"$and": [{"category_code": 'games_video'}, {"offices.latitude": {"$not": re.compile("^N.*")}}, {"founded_year": {"$gte": 1965}} ]}).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(1000)

gamesvideo_co = list(map(lambda x: [x["name"], x["offices"], x["category_code"]], list(cursor3)))

gamesvideo = pd.DataFrame(gamesvideo_co)

In [53]:
#Preguntamos a mongo por 1000 compañías de web para sacar su lat y long.
#Web Co

cursor4 = db.companies.find({"$and": [{"category_code": 'web'}, {"offices.latitude": {"$not": re.compile("^N.*")}}, {"founded_year": {"$gte": 1965}} ]}).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(1000)

web_co = list(map(lambda x: [x["name"], x["offices"], x["category_code"]], list(cursor4)))

web = pd.DataFrame(web_co)


In [54]:
cos = pd.concat([software, mobile, gamesvideo, web])
cos.columns = ['Company', 'Address', 'Category']
print(len(cos))

3384


In [79]:
cos = cos.dropna(subset=['Address'], axis=0)
print(len(cos))
cos.head()

3384


,Company,Address,Category
0,BMC Software,"[{'description': '', 'address1': '2101 City We...",software
1,Rackspace,"[{'description': 'HQ', 'address1': '5000 Walze...",software
2,Jive Software,"[{'description': 'Palo Alto HQ', 'address1': '...",software
3,QlikTech,"[{'description': 'World Headquarters', 'addres...",software
4,Cornerstone OnDemand,"[{'description': 'Headquarters', 'address1': '...",software


In [ ]:
'''
lats = []
lons = []
for y in listadir:
    for q in y:
        lat, lon = q.get('latitude'), q.get('longitude')
        # print(lat, lon)
        lats.append(lat)
        lons.append(lon)
            
     #No me deja iterar sobre un diccionario, solo me deja print it  (esto de abajo no me lo deja hacer)      
            
    #latlong.append(y.get('latitude'))
    #latlong.append(y.get('longitude'))
    
# for y in listadir:
  #  for q in y:
      #  q.values()
        #del q['address1']
        #q.pop('address1', None)
        
#print(q)

cos['latitude'] = lats
cos['longitude'] = lons

cos.head(20)
'''

In [80]:
# Limpiar columna Address para dejar coordenadas

# Queremos tener en cuenta todas las direcciones de TODAS las oficinas de cada empresa.

# cos.iloc[0]['Address']
# cos.iloc[0]['Address'][0]['latitude']

l = []
names = []
categories = []

for i in range(3384):
    norm = json_normalize(cos.iloc[i]['Address'])
    l.append(norm)
    
    name = cos.iloc[i]['Company']
    
    for n in range(len(norm)): 
        names.append(name)
    
    categori = cos.iloc[i]['Category']
    
    for c in range(len(norm)):
        categories.append(categori)

# print(l)

In [134]:
wa = pd.concat(l)
wa['Company'] = names
wa['Category'] = categories
#wa

In [135]:
wa = wa.drop(columns=['address1', 'address2', 'city', 'country_code', 'description', 'state_code', 'zip_code'])
wa = wa.dropna(subset=['longitude'], axis=0)
print(len(wa))
wa.head()

2549


,latitude,longitude,Company,Category
0,29.780549,-95.345792,BMC Software,software
0,29.527664,-98.562993,Rackspace,software
0,40.037578,-75.366299,QlikTech,software
0,34.028182,-118.472577,Cornerstone OnDemand,software
0,30.245535,-97.805621,SolarWinds,software


In [136]:
points = []
for i in range(len(wa)): 
    lat = wa.iloc[i]['latitude']
    lon = wa.iloc[i]['longitude']
    point = {"type":"Point", "coordinates": [lon,lat]}
    points.append(point)

wa['points'] = points
wa.head()

,latitude,longitude,Company,Category,points
0,29.780549,-95.345792,BMC Software,software,"{'type': 'Point', 'coordinates': [-95.3457921,..."
0,29.527664,-98.562993,Rackspace,software,"{'type': 'Point', 'coordinates': [-98.562993, ..."
0,40.037578,-75.366299,QlikTech,software,"{'type': 'Point', 'coordinates': [-75.366299, ..."
0,34.028182,-118.472577,Cornerstone OnDemand,software,"{'type': 'Point', 'coordinates': [-118.472577,..."
0,30.245535,-97.805621,SolarWinds,software,"{'type': 'Point', 'coordinates': [-97.805621, ..."


In [137]:
wa

,latitude,longitude,Company,Category,points
0,29.780549,-95.345792,BMC Software,software,"{'type': 'Point', 'coordinates': [-95.3457921,..."
0,29.527664,-98.562993,Rackspace,software,"{'type': 'Point', 'coordinates': [-98.562993, ..."
0,40.037578,-75.366299,QlikTech,software,"{'type': 'Point', 'coordinates': [-75.366299, ..."
0,34.028182,-118.472577,Cornerstone OnDemand,software,"{'type': 'Point', 'coordinates': [-118.472577,..."
0,30.245535,-97.805621,SolarWinds,software,"{'type': 'Point', 'coordinates': [-97.805621, ..."
0,37.558300,-122.301385,Marketo,software,"{'type': 'Point', 'coordinates': [-122.301385,..."
0,37.498842,-122.239815,DemandTec,software,"{'type': 'Point', 'coordinates': [-122.239815,..."
1,51.506921,-0.124934,DemandTec,software,"{'type': 'Point', 'coordinates': [-0.1249337, ..."
0,40.755001,-73.971736,Vringo,software,"{'type': 'Point', 'coordinates': [-73.9717364,..."
0,37.692934,-121.904945,Zoho,software,"{'type': 'Point', 'coordinates': [-121.904945,..."


In [93]:
wa.to_json("cos_with_coords.json", orient="records", lines=True)

In [94]:
# Ahora, en Terminal hay que cargar este Json en MongoDB para poner la condición $near, 
# mongoimport --db companies --collection clean  --file cos_with_coords.json (--drop así solo si quisieramos borrar el anterior)
# después hay que crear un index en el clean collection de compass, `2d sphere` index  with the `geojson` document key (Usar points de key en vez de points.coord)

In [138]:
client = MongoClient()
db = client.companies
collection = db.clean

co_coords = list(collection.find({"Company":{"$exists":"true"}},{"Company":1, '_id':0, "points.coordinates":1}))

def find_near(i):
    lat = co_coords[i]["points"]["coordinates"][0]
    lgtd = co_coords[i]["points"]["coordinates"][1]

    local_cursor = collection.find({"points": {"$near": {"$geometry": {
                                                        "type": "Point" ,
                                                        "coordinates": [ lat , lgtd ]},
                                          "$minDistance": 0,
                                          "$maxDistance": 1000}}})
    return list(local_cursor)

near_list = [find_near(i) for i in range(len(co_coords))]
near_count = [len(e) for e in near_list]
#co_coords[i]["points"]["coordinates"][0]

In [139]:
# El máximo nos dice la empresa que más empresas tiene cerca, hay una empresa que tiene 88 empresas que nos interesan cerca

max(near_count)

88

In [140]:
wa["Number of close companies"] = near_count
wa.head()

,latitude,longitude,Company,Category,points,Number of close companies
0,29.780549,-95.345792,BMC Software,software,"{'type': 'Point', 'coordinates': [-95.3457921,...",21
0,29.527664,-98.562993,Rackspace,software,"{'type': 'Point', 'coordinates': [-98.562993, ...",1
0,40.037578,-75.366299,QlikTech,software,"{'type': 'Point', 'coordinates': [-75.366299, ...",1
0,34.028182,-118.472577,Cornerstone OnDemand,software,"{'type': 'Point', 'coordinates': [-118.472577,...",17
0,30.245535,-97.805621,SolarWinds,software,"{'type': 'Point', 'coordinates': [-97.805621, ...",2


In [144]:
wa.sort_values(by = "Number of close companies", ascending = False)

,latitude,longitude,Company,Category,points,Number of close companies
0,37.791391,-122.402859,Overstream,games_video,"{'type': 'Point', 'coordinates': [-122.402859,...",88
0,37.787092,-122.399972,Curse,games_video,"{'type': 'Point', 'coordinates': [-122.399972,...",84
0,38.947914,-77.342896,Axiom Software,software,"{'type': 'Point', 'coordinates': [-77.342896, ...",83
1,41.073034,-73.550452,InstallFree,software,"{'type': 'Point', 'coordinates': [-73.550452, ...",83
0,34.089679,-118.292618,CrewedUp,web,"{'type': 'Point', 'coordinates': [-118.292618,...",83
0,37.351894,-122.085807,SezWho,web,"{'type': 'Point', 'coordinates': [-122.085807,...",83
0,40.738476,-73.998726,VISUALPLANT,games_video,"{'type': 'Point', 'coordinates': [-73.998726, ...",83
0,31.046051,34.851612,KnockaTV,games_video,"{'type': 'Point', 'coordinates': [34.851612, 3...",82
0,37.781002,-122.406912,Tiny Pictures,mobile,"{'type': 'Point', 'coordinates': [-122.406912,...",82
0,40.569222,-111.862460,Computer Hope,web,"{'type': 'Point', 'coordinates': [-111.86246, ...",82
